In [2]:
import pandas as pd
import matplotlib.pyplot as plt

In [3]:
base_path = "/media/bruno/Arquivos/Desenvolvimento/UBS"
dataset = pd.read_csv(f"{base_path}/data/dataset.csv")

In [4]:
dataset

,UBS_LON,UBS_LAT,UF,CITY,CITY_LON,CITY_LAT
0,-52.22545,-15.90682,GO,Aragarças,-52.25,-15.89
1,-35.03210,-8.28389,PE,Cabo de Santo Agostinho,-35.03,-8.28
2,-35.02819,-8.28353,PE,Cabo de Santo Agostinho,-35.03,-8.28
3,-35.03500,-8.28700,PE,Cabo de Santo Agostinho,-35.03,-8.28
4,-35.03210,-8.28389,PE,Cabo de Santo Agostinho,-35.03,-8.28
...,...,...,...,...,...,...
41077,-42.53447,-2.71321,MA,Paulino Neves,-42.53,-2.71
41078,-42.15539,-20.83200,MG,Pedra Dourada,-42.15,-20.82
41079,-44.73800,-2.70600,MA,Bacurituba,-44.73,-2.70
41080,-41.00047,-17.43297,MG,Pavão,-40.99,-17.42
